## Simple Reflection Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph, MessageGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os

from IPython.display import Image, display

import concurrent.futures

In [28]:
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o",
)

class PromptUpdate(BaseModel):
    """Review of the prompt"""
    updated_prompt: str = Field(description="Updated prompt based on the review")
    justification: str = Field(description="Justification for the changes made")

def self_reflection_graph(additional_info) -> MessageGraph:
    """
    Constructs a graph for self-reflection and improvement of prompts.
    """

    system_message = SystemMessage(content=f"""You are an experienced: AI engineer. Your role: Analyze prompts and provide feedback. Your function: Determine how the prompt can be improved by utilising knowledge of large language models and considering best practices.
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")

    def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptUpdate)
        prompt_text = """Your task is to improve the prompt in light of your role, position, and function.
If the user provides critique and recommendations for the prompt, respond with a revised version of your previous attempts.
You must provide justification for any changes you make to the prompt. Your justification must be less than 50 words.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow if making changes to the prompt:
1. DO NOT modify existing restrictions in the prompt.
2. DO NOT modify or remove negations in the prompt.
3. DO add additional restrictions or negations if you deem them necessary.
4. DO NOT modify or remove placeholders in the prompt.
5. DO NOT add additional placeholders.
6. DO add additional instructions if you deem them necessary.
7. DO re-format the prompt if you deem it necessary (for example, adding line breaks, bullet points, etc.).

Your update process should be as follows:
- Read the prompt carefully asn an expert AI engineer.
- Think carefully about how YOU specifically can contribute to the improvement of the prompt.
- Make changes to the prompt as you see fit.
- Ensure that your changes adhere to the strict guidelines.
- Provide justification for the changes you made.
- Ensure that your justification is less than 50 words.
- Submit your updated prompt and justification.

Return the updated prompt along with your review in JSON format below. You must not output your review.

{{
"updated_prompt": "updated prompt based on the review",
"justification": "Justification for the changes made"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly.
"""
        prompt = HumanMessagePromptTemplate(
            prompt = PromptTemplate(
                system_message=system_message,
                template=prompt_text,
                input_variables=["additional_info"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
            )
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                prompt,
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        for _ in range(3):
            try:
                generate = prompt | llm | pydantic_parser
                res = generate.invoke({"messages": state, "additional_info": additional_info})
                updated_prompt_message = AIMessage(content=res.updated_prompt)
                justification_message = AIMessage(content=res.justification)
                new_state = state + [updated_prompt_message, justification_message]
                print(f"New state: {new_state}")
                return new_state
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")
        
    def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
        reflection_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    f"""{system_message.content} 
Your task is to evaluate the user's prompt and provide feedback on how it can be improved in light of your role, position, and function.
Your feedback must be less than 50 words so think carefully about the most critical aspects of the prompt that need improvement in light of your position, role and function.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
1. DO NOT suggest modifications to restrictions.
2. DO NOT suggest modifications to or removal of negations.
3. DO suggest additional restrictions or negations if you deem them necessary.
4. DO NOT suggest modifications to or removal of placeholders.
5. DO NOT suggest additional placeholders.
6. DO suggest additional instructions if you deem them necessary.
7. DO suggest re-formatting the prompt if you deem it necessary (for example, adding line breaks, bullet points, etc.).

Your reviewal process should be as follows:
- Read the prompt carefully as an expert AI engineer. 
- Identify the most critical aspects of the prompt that need improvement in light of your position, role and function.
- Provide feedback and recommendations on how to improve the prompt in light of your position, role and function.
- Ensure that your feedback is less than 50 words.
- Ensure that your feedback follows the strict guidelines provided above.
- Submit your feedback.
""",
                ),
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        reflect = reflection_prompt | llm
        res = reflect.invoke({"messages": messages})
        # We treat the output of this as human feedback for the generator
        return HumanMessage(content=res.content)

    builder = MessageGraph()
    builder.add_node("generate", generation_node)
    builder.add_node("reflect", reflection_node)
    builder.set_entry_point("generate")

    def should_continue(state: List[BaseMessage]):
        if len(state)/2 > 10:
            return END
        return "reflect"

    builder.add_conditional_edges("generate", should_continue)
    builder.add_edge("reflect", "generate")
    graph = builder.compile()

    return graph

def update_prompt(prompt: str, additional_info: str) -> str:
    """
    Uses self_reflection_graph to iteratively act on feedback and update prompt
    """
    graph = self_reflection_graph(additional_info)
    inputs = [HumanMessage(content=prompt)]
    for event in graph.stream(
        inputs,
        {"recursion_limit": 100}
    ):
        print(event)
        print("----")
    return event

In [29]:
base_prompt = "{content}. Please output your answer at the end as ##<your answer (arabic numerals)>."
additional_info = """1. Solve the math word problem. 2. Output the correct answer at the end as ##<your answer (arabic numerals)> with no spaces or units."""

prompt = update_prompt(base_prompt, additional_info)

New state: [HumanMessage(content='{content}. Please output your answer at the end as ##<your answer (arabic numerals)>.', id='f3a0fa58-26bc-4c2f-93be-2155496b9435'), AIMessage(content='Solve the math word problem. Please output your answer at the end as ##<your answer (arabic numerals)> with no spaces or units.'), AIMessage(content='Added the instruction to solve the math word problem explicitly to ensure clarity and adherence to the task requirements.')]
{'generate': [HumanMessage(content='{content}. Please output your answer at the end as ##<your answer (arabic numerals)>.', id='f3a0fa58-26bc-4c2f-93be-2155496b9435'), AIMessage(content='Solve the math word problem. Please output your answer at the end as ##<your answer (arabic numerals)> with no spaces or units.', id='d217522c-173c-4b5e-a3d7-1bb70ece02fc'), AIMessage(content='Added the instruction to solve the math word problem explicitly to ensure clarity and adherence to the task requirements.', id='455cc43c-3785-4ebf-872a-b12f0914

In [30]:
prompt["generate"][-2].content

'Solve the math word problem. Please output your answer at the end as ##<your answer (arabic numerals)> with no spaces or units.'